In [1]:
#import cv2
import pandas as pd
import numpy as np
import sys



sys.path.append("../src")

import make_camera_coordinates as mcc

get the rotation matricies for all points in C looking at all points in P

    INPUTS
    ------
    C: numpy array of shape (num camera points, 3)
    P: numpy array of shape (num target points, 3)

    OUTPUTS
    -------

    extrinsic_camera_matrix : numpy array of shape (num target points, num camera points, 3, 3)

    algorithm for look-at camera rotation matrix
    1. Compute L = p - C.
    2. Normalize L.
    3. Compute s = L x u. (cross product)
    4. Normalize s.
    5. Compute u_ = s x L.
    6. Then Extrinsic rotation matrix given by:

            s1,  s2,  s3
    R =     u_1, u_2, u_3
            -L1, -L2, -L3

    u is the y-axis -- vary this to get roll

    This takes advantage of broadcasting 
        -- see https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html

In [2]:

# point we want to project
b = np.array([3,0,0,1])

# target point
p = np.array([9 , -2,  0.    ])

# camera center in world coordinates
C = np.array([ 0 ,   0.,  15.24  ])

# up vector
u = np.array([0,1,0], dtype='float32')


#inrinsic parameters

# physical sensor width and hiehgt in iphone in real life in m
F  = 4.11/1000. # focal length
W = 4.8/1000.
H = 3.6 / 1000.  
w = 1280. # width in pixels
h = 720 # height in pixels
resolution = (w, h)

fy=1229.; cy=360.; fx=1153.; cx=640.;
fy=4.11/1000.; cy=0; fx=4.11/1000.; cx=0;
fy=F*h/H; cy=0; fx=F*w/W; cx=0;

K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])

In [3]:
# rotation matrix
L = p - C
print "L after sub: %s" % L
L = L / np.linalg.norm(L)
print "L after norm: %s" % L
s = np.cross(L, u)
s = s / np.linalg.norm(s)
print "s after norm: %s" % s
u_ = np.cross(s, L)
print "u_: %s" % u_
R = np.vstack([s, u_, -L])
print "R before fiddle:\n %s" % R
    
# transformation vector
R[0] = -R[0]
R[2] = -R[2]
print "final R:\n%s" % R
t = -np.dot(R, C)
print "t: %s" % t

extrinsic_matrix = np.hstack([R, t.reshape(-1,1)])
print "extrinsic_matrix:\n %s" % extrinsic_matrix

L after sub: [  9.    -2.   -15.24]
L after norm: [ 0.5052851  -0.11228558 -0.8556161 ]
s after norm: [ 0.86106148 -0.          0.50850087]
u_: [ 0.05709731  0.99367598 -0.09668479]
R before fiddle:
 [[ 0.86106148 -0.          0.50850087]
 [ 0.05709731  0.99367598 -0.09668479]
 [-0.5052851   0.11228558  0.8556161 ]]
final R:
[[-0.86106148  0.         -0.50850087]
 [ 0.05709731  0.99367598 -0.09668479]
 [ 0.5052851  -0.11228558 -0.8556161 ]]
t: [  7.74955328   1.47347613  13.03958942]
extrinsic_matrix:
 [[ -0.86106148   0.          -0.50850087   7.74955328]
 [  0.05709731   0.99367598  -0.09668479   1.47347613]
 [  0.5052851   -0.11228558  -0.8556161   13.03958942]]


In [4]:
print "C vector: position of camera in world coordinates"
print C
print "*******"
print "t vector, position of world origin in camera coordinates"
print t
print "*******"
b_cam = np.dot(extrinsic_matrix, b)
print "B_cam: the location of the point in camera coordinates"
print b_cam
print "*******"
print "extrinsic_matrix is "
print  extrinsic_matrix
print "*******"
print "K is the camera matrix"
print K
print "*******"
P = np.dot(K, extrinsic_matrix)
print "P is: camera matrix"
print P
print "*******"
print "we are looking from:"
print C
print "*******"
print "the point we are projecting is:"
print b
print "we are looking at:"
print p
# print "the projection into the camera is:"
# print np.dot(P, b)

C vector: position of camera in world coordinates
[  0.     0.    15.24]
*******
t vector, position of world origin in camera coordinates
[  7.74955328   1.47347613  13.03958942]
*******
B_cam: the location of the point in camera coordinates
[  5.16636885   1.64476807  14.55544472]
*******
extrinsic_matrix is 
[[ -0.86106148   0.          -0.50850087   7.74955328]
 [  0.05709731   0.99367598  -0.09668479   1.47347613]
 [  0.5052851   -0.11228558  -0.8556161   13.03958942]]
*******
K is the camera matrix
[[  1.09600000e+03   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   8.22000000e+02   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]
*******
P is: camera matrix
[[ -9.43723377e+02   0.00000000e+00  -5.57316955e+02   8.49351039e+03]
 [  4.69339922e+01   8.16801654e+02  -7.94748935e+01   1.21119738e+03]
 [  5.05285100e-01  -1.12285578e-01  -8.55616103e-01   1.30395894e+01]]
*******
we are looking from:
[  0.     0.    15.24]
*******
the point we are projectin

In [5]:
#using this wiki article: https://en.wikipedia.org/wiki/Pinhole_camera_model
# 4.11 comes from 4.11 mm focal length (in real world terms) of iphone 6 
#    -- see http://photo.stackexchange.com/questions/57560/so-my-iphone-6-camera-lens-is-as-wide-as-my-full-frame-35mm-dslr-lens

# TODO: add aspect ratio
f  = 4.11/1000.
W = 4.8/1000.
H = 3.6 / 1000.   # http://photoseek.com/2013/compare-digital-camera-sensor-sizes-full-frame-35mm-aps-c-micro-four-thirds-1-inch-type/
y = b_cam[0:2]*(f / b_cam[2])

# flip again because it will be on the other side
resolution =np.array((1280, 720))
w, h = resolution
print "THIS WORKS\n"
# actual length in meters
print "actual length in meters"
print y
print "\n divide by sensor width:"
print "\nlength in terms of percent of screen"
pct_scr = y / W
print pct_scr
print "\n multiply by number of pixels wide the image is"
print "\nlength in terms of pixels"
relative_pixels_from_center = pct_scr * resolution
print relative_pixels_from_center
#print - projection / sensor_width
print "\n add half of the image pixels wide"
print '\nour projection is:'
print relative_pixels_from_center + resolution/2

THIS WORKS

actual length in meters
[ 0.00145882  0.00046443]

 divide by sensor width:

length in terms of percent of screen
[ 0.30392086  0.09675642]

 multiply by number of pixels wide the image is

length in terms of pixels
[ 389.01870545   69.66461935]

 add half of the image pixels wide

our projection is:
[ 1029.01870545   429.66461935]


In [29]:
#replicate with matrix way

# load up matricies
with open("../src/extrinsic_matrix.npy", "r") as fil:
    openfil = np.load(fil)
    camera_points = openfil['camera_points']
    lookat_points = openfil['lookat_points']
    extrinsic_matrix = openfil['extrinsic_matrix']
    
with open("../src/results.npz", "r") as fil2:
    openfil = np.load(fil2)
    results = openfil['results']
    info = openfil['info']

In [30]:
print results.shape
print extrinsic_matrix.shape
print lookat_points.shape
print camera_points.shape

(612, 288, 4)
(36, 459, 3, 4)
(36, 3)
(459, 3)


In [31]:
## THESE are the x, y coordinates for all pitches
pitch_points = results[:,:,1:3]
print pitch_points.shape

(612, 288, 2)


In [33]:
# we append on 0 for the z axis because all of our pitches are in the z axis, and then 1 for homogenous coordinates
to_concat = np.zeros(pitch_points.shape, dtype=pitch_points.dtype)
to_concat[:,:,1] = 1.
pitch_points_homo = np.concatenate([pitch_points, to_concat], axis=2)

print pitch_points.shape
print pitch_points_homo.shape
assert pitch_points_homo[:,:,2:4].sum()==np.product(pitch_points_homo.shape[0:2])

(612, 288, 2)
(612, 288, 4)


In [34]:
camera_index=0
lookat_index = 0
pitch=0
frame=range(5)
em = extrinsic_matrix[lookat_index,camera_index]
print "The shape of the extrinsic matrix is:"
print em.shape

print "\nThe camera point is:"
print camera_points[camera_index]
print "\nThe target point index is:"
print lookat_points[lookat_index]
print "\nThe ball points are at:"

print pitch_points_homo[pitch, frame]


The shape of the extrinsic matrix is:
(3, 4)

The camera point is:
[ -0.3048       1.21920002 -30.47999954]

The target point index is:
[ 0.         -0.60960001  0.        ]

The ball points are at:
[[ 0.          1.52400005  0.          1.        ]
 [ 0.0930766   1.52066457  0.          1.        ]
 [ 0.18615159  1.5171591   0.          1.        ]
 [ 0.27922338  1.51348364  0.          1.        ]
 [ 0.37229192  1.50963819  0.          1.        ]]


In [35]:
print "The ball points in camera coordinates"
#print em
#print em.shape
b_cam = np.dot(em, pitch_points_homo[pitch,frame].T).T
print b_cam
print b_cam.shape

The ball points in camera coordinates
[[  1.44816426e-09   2.12977004e+00   3.04085541e+01]
 [  9.30719450e-02   2.12649655e+00   3.04096813e+01]
 [  1.86142281e-01   2.12305284e+00   3.04108219e+01]
 [  2.79209405e-01   2.11943984e+00   3.04119701e+01]
 [  3.72273296e-01   2.11565709e+00   3.04131298e+01]]
(5, 3)


In [36]:
print extrinsic_matrix.shape
print pitch_points_homo.shape

(36, 459, 3, 4)
(612, 288, 4)


In [37]:
pitch_points_homo[pitch,frame].T.shape

(4, 5)

In [55]:
em2 = extrinsic_matrix[:,0,:,:]
print em2.shape
print pitch_points_homo[0,:,:].shape

(36, 3, 4)
(288, 4)


In [56]:
np.tensordot(pitch_points_homo[0,:,:], em2, axes=([1],[2])).shape

(288, 36, 3)

### THIS IS PROOF THAT THE TENSORDOT WORKS

In [40]:
results_temp = np.tensordot(pitch_points_homo[0:10,0:2,:], em2, axes=([2], [3]))
print results_temp.shape

IndexError: tuple index out of range

In [17]:
for i1 in range(10):
    for j1 in range(2):
        for i2 in range(2):
            for j2 in range(3):
                #print "%s %s %s %s" % (i1, j1, i2, j2)
                #print np.dot(em2[0,0], pitch_points_homo[0,0,:])
                print np.all(np.isclose(results_temp[i1,j1,i2,j2], np.dot(em2[i2,j2], pitch_points_homo[i1,j1,:])))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [22]:
em = extrinsic_matrix[:50, 0:3,:,:]
b_cam = np.tensordot(pitch_points_homo, em, axes=([2], [3]))
print b_cam.shape
print b_cam.nbytes

(5436, 288, 50, 3, 3)
2818022400


In [271]:
print extrinsic_matrix.shape

(403, 28728, 3, 4)


In [185]:
f  = 4.11/1000.
W = 4.8/1000.
H = 3.6 / 1000.   # http://photoseek.com/2013/compare-digital-camera-sensor-sizes-full-frame-35mm-aps-c-micro-four-thirds-1-inch-type/
c_dim = np.array([W, H])
y = b_cam[:,0:2]*(f / b_cam[:,2]).reshape(-1,1)
# TODO: add aspect ratio

# flip again because it will be on the other side
resolution =np.array((1280, 720))
w, h = resolution
print "THIS WORKS\n"
# actual length in meters
print "actual length in meters"
print y
print "\n divide by sensor width:"
print "\nlength in terms of percent of screen"
pct_scr = y / c_dim
print pct_scr
print "\n multiply by number of pixels wide the image is"
print "\nlength in terms of pixels"
relative_pixels_from_center = pct_scr * resolution
print relative_pixels_from_center
#print - projection / sensor_width
print "\nadd half of the image pixels wide"
print '\nour projection is:'
print relative_pixels_from_center + resolution/2

THIS WORKS

actual length in meters
[[  1.05837078e-13   1.11986839e-04]
 [  3.41695772e-06   1.11861445e-04]
 [  6.83360440e-06   1.11725618e-04]
 [  1.02498943e-05   1.11579393e-04]
 [  1.36658227e-05   1.11422756e-04]]

 divide by sensor width:

length in terms of percent of screen
[[  2.20493913e-11   3.11074554e-02]
 [  7.11866193e-04   3.10726237e-02]
 [  1.42366758e-03   3.10348939e-02]
 [  2.13539465e-03   3.09942758e-02]
 [  2.84704640e-03   3.09507656e-02]]

 multiply by number of pixels wide the image is

length in terms of pixels
[[  2.82232209e-08   2.23973679e+01]
 [  9.11188727e-01   2.23722891e+01]
 [  1.82229451e+00   2.23451236e+01]
 [  2.73330515e+00   2.23158786e+01]
 [  3.64421940e+00   2.22845512e+01]]

add half of the image pixels wide

our projection is:
[[ 640.00000003  382.39736787]
 [ 640.91118873  382.3722891 ]
 [ 641.82229451  382.34512358]
 [ 642.73330515  382.3158786 ]
 [ 643.6442194   382.28455123]]


In [73]:
### this is saved as an archive

f  = 4.11/1000.
W = 4.8/1000.
H = 3.6 / 1000.   # http://photoseek.com/2013/compare-digital-camera-sensor-sizes-full-frame-35mm-aps-c-micro-four-thirds-1-inch-type/

y = b_cam[:,0:2]*(f / b_cam[:,2])
# TODO: add aspect ratio

# flip again because it will be on the other side
resolution =np.array((1280, 640))
w, h = resolution
print "THIS WORKS\n"
# actual length in meters
print "actual length in meters"
print y
print "\n divide by sensor width:"
print "\nlength in terms of percent of screen"
pct_scr = y / W
print pct_scr
print "\n multiply by number of pixels wide the image is"
print "\nlength in terms of pixels"
relative_pixels_from_center = pct_scr * resolution
print relative_pixels_from_center
#print - projection / sensor_width
print "\n add half of the image pixels wide"
print '\nour projection is:'
print relative_pixels_from_center + resolution/2

THIS WORKS

actual length in meters
[ -5.78100935e-11   4.84554184e-04]

 divide by sensor width:

length in terms of percent of screen
[ -1.20437686e-08   1.00948781e-01]

 multiply by number of pixels wide the image is

length in terms of pixels
[ -1.54160239e-05   6.46072197e+01]

 add half of the image pixels wide

our projection is:
[ 639.99998458  384.6072197 ]


In [63]:
print info[0]
print camera_points[0]
print lookat_points[0]

[  0.00000000e+00   1.21920002e+00  -2.00000000e+00   1.34111996e+01
   1.20546587e+00   3.00000000e-01   4.38024598e-03   1.45290000e-01
   4.16666667e-03   2.88000000e+02]
[ -3.0480001    0.91439998  15.23999977]
[ 0.         -0.60960001  0.        ]


In [185]:
print intrinsic_matrix
print np.dot(intrinsic_matrix, wc)

[[1153    0  640]
 [   0 1229  360]
 [   0    0    1]]
[[ 55792.47547326]
 [ 35912.1228171 ]
 [    99.75589671]]


In [186]:
toScreen = np.array([[640, 0,0, 640],[0, -360,0, 360], [0, 0, 1, 0], [0, 0, 0,1]])
print toScreen

[[ 640    0    0  640]
 [   0 -360    0  360]
 [   0    0    1    0]
 [   0    0    0    1]]


In [187]:
# this seems to wrok
np.dot(toScreen, np.vstack([wc,[1]]))

array([[ -3.82906417e+03],
       [  3.60000000e+02],
       [  9.97558967e+01],
       [  1.00000000e+00]])

In [129]:
-1.19902647e+03

-1199.02647

In [81]:
np.linalg.det(R)

0.99999999999999989

In [149]:
def rotate_x(theta_x):
    r1 = [1,0,0,0]
    r2 = [0, np.cos(theta_x), -np.sin(theta_x), 0]
    
    r3 = [0, np.sin(theta_x), np.cos(theta_x), 0]
    
    r4 = [0,0,0,1]
    return np.array([r1,r2,r3,r4])

def rotate_y(theta_y):
    r1 = [np.cos(theta_y), 0, np.sin(theta_y), 0]
    r2 = [0, 1, 0, 0]
    
    r3 = [-np.sin(theta_y),0, np.cos(theta_y), 0]
    
    r4 = [0,0,0,1]
    return np.array([r1,r2,r3,r4])

def rotate_z(theta_z):
    r1 = [np.cos(theta_z), -np.sin(theta_z), 0, 0]
    r2 = [np.sin(theta_z), np.cos(theta_z), 0, 0 ]
    
    r3 = [0, 0, 1, 0]
    
    r4 = [0, 0, 0, 1]
    return np.array([r1,r2,r3,r4])

def rotate(p, thetas=(0,0,0), degrees=False, transpose=False):
    """ rotate x, then y, then z"""
    if degrees == True:
        thetas = [np.deg2rad(th) for th in thetas]
    X_rot_mat = rotate_x(thetas[0])
    Y_rot_mat = rotate_y(thetas[1])
    Z_rot_mat = rotate_z(thetas[2])
    if transpose:
        X_rot_mat = X_rot_mat.T
        Y_rot_mat = Y_rot_mat.T
        Z_rot_mat = Z_rot_mat.T
        
    rot_mat = np.dot(np.dot(Z_rot_mat, Y_rot_mat), X_rot_mat)
    return np.dot(rot_mat, p), rot_mat

    #return np.dot(np.dot(np.dot(rotate_x(thetas[0]), rotate_y(thetas[1])), rotate_z(thetas[2])), p)

def CtoT(C):
    """C is camera position, will return translation matrix"""
    
    return np.vstack([np.hstack([np.identity(3), C.reshape(-1,1)]), np.array([0,0,0,1])])


In [18]:
np.array([1,1,0,1])*rotate_z(np.deg2rad(45))


array([[ 0.70710678, -0.70710678,  0.        ,  0.        ],
       [ 0.70710678,  0.70710678,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [21]:
np.dot(rotate_y(np.deg2rad(45)), np.array([1,0,0,1]))


array([ 0.70710678,  0.        , -0.70710678,  1.        ])

In [121]:
p1, R = rotate(np.array([1,0,0,1]),thetas=(0,45, 45), degrees=True,transpose=False)

In [122]:
p1

array([ 0.5       ,  0.5       , -0.70710678,  1.        ])

In [123]:
R

array([[ 0.5       , -0.70710678,  0.5       ,  0.        ],
       [ 0.5       ,  0.70710678,  0.5       ,  0.        ],
       [-0.70710678,  0.        ,  0.70710678,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [150]:
C = np.array([0,0,5]) # position of camera

Tr = CtoT(C)
Tr

array([[ 1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  5.],
       [ 0.,  0.,  0.,  1.]])

In [151]:
T = np.dot(R, Tr)

In [152]:
T

array([[ 0.5       , -0.70710678,  0.5       ,  2.5       ],
       [ 0.5       ,  0.70710678,  0.5       ,  2.5       ],
       [-0.70710678,  0.        ,  0.70710678,  3.53553391],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [153]:
np.dot(T,np.array([1,0,0,1]))

array([ 3.        ,  3.        ,  2.82842712,  1.        ])

In [154]:
p1

array([ 0.5       ,  0.5       , -0.70710678,  1.        ])

In [155]:
np.dot(Tr, p1)

array([ 0.5       ,  0.5       ,  4.29289322,  1.        ])

In [156]:
np.dot(Tr, np.array([1,0,0,1]))

array([ 1.,  0.,  5.,  1.])